# Условия задачи:

### У Вас есть ассоциативные правила (edges) для рекомендаций товаров (item-item). Ваша задача написать метод, который будет выдавать ранжированный список из 4 наиболее релевантных рекомендуемых к корзине товаров. Для этого вам необходимо объединить рекомендации для всех товаров корзины и, если необходимо, дополнить их наиболее популярными товарами (частоты покупки freqs). При этом часть товаров недоступна в момент покупки (absent), такие товары рекомендовать нельзя.

# Решение:

### Импорт библиотеки

In [68]:
import pandas as pd

### Данные ассоциативных правил (edges)

In [69]:
edges = [
    [1,2,300],
    [1,4,150],
    [1,7,220],
    [2, 1, 100],
    [2, 5, 520],
    [2, 10, 110],
    [3, 4, 340],
    [4, 1, 150],
    [4, 3, 340],
    [5, 2, 520],
    [7, 1, 220],
    [9, 10, 230],
    [10, 2, 110],
    [10, 9, 230]
    
]

edges = pd.DataFrame(edges, columns=['a_id', 'b_id', 'score'])
edges #Для удобства перевели в Датафрейм

,a_id,b_id,score
0,1,2,300
1,1,4,150
2,1,7,220
3,2,1,100
4,2,5,520
5,2,10,110
6,3,4,340
7,4,1,150
8,4,3,340
9,5,2,520


### "Бестселлеры" (freqs)

In [70]:
freqs = [
    [1, 1234],
    [2, 1505],
    [3, 900],
    [4, 2345],
    [5, 378],
    [6, 2998],
    [7, 5421],
    [8, 1323],
    [9, 708],
    [10, 1283]
]
bestsellers = [i[0] for i in sorted(freqs, key=lambda x: x[1], reverse=True)]
bestsellers

[7, 6, 4, 2, 8, 10, 1, 3, 9, 5]

### Товары которых нет в наличии (absent)  - их не рекомендуем, корзина товаров покупателей (basket), всего 6 клиентов

In [71]:
absent = [5, 1, 10]

basket = [
    [5, 10, 8, 9],
    [8, 2, 7],
    [4, 8, 1, 9, 7],
    [10,],
    [6,]
]

In [72]:
#Для удобства добавим пронумерованных клиентов и переведем в ДатаФрейм
basket = [['client_'+str(i+1), j] for i,j in enumerate(basket)]
basket

[['client_1', [5, 10, 8, 9]],
 ['client_2', [8, 2, 7]],
 ['client_3', [4, 8, 1, 9, 7]],
 ['client_4', [10]],
 ['client_5', [6]]]

In [73]:
basket = pd.DataFrame(basket, columns=['client_id','item_id'])
basket.head()

,client_id,item_id
0,client_1,"[5, 10, 8, 9]"
1,client_2,"[8, 2, 7]"
2,client_3,"[4, 8, 1, 9, 7]"
3,client_4,[10]
4,client_5,[6]


In [74]:
#Переведём наш ДатаФрейм из широкого вида в длинный вид 
basket = basket.explode('item_id')
basket

,client_id,item_id
0,client_1,5
0,client_1,10
0,client_1,8
0,client_1,9
1,client_2,8
1,client_2,2
1,client_2,7
2,client_3,4
2,client_3,8
2,client_3,1


In [75]:
#Джойним ассоциативные правила к каждому товару наших покупателей 
basket_with_scores = basket.merge(edges, left_on='item_id', right_on='a_id')
basket_with_scores

,client_id,item_id,a_id,b_id,score
0,client_1,5,5,2,520
1,client_1,10,10,2,110
2,client_1,10,10,9,230
3,client_4,10,10,2,110
4,client_4,10,10,9,230
5,client_1,9,9,10,230
6,client_3,9,9,10,230
7,client_2,2,2,1,100
8,client_2,2,2,5,520
9,client_2,2,2,10,110


In [76]:
# фильтрация - удаляем товары, которых нет в наличии
basket_with_scores = basket_with_scores[~basket_with_scores['b_id'].isin(absent)]
basket_with_scores

,client_id,item_id,a_id,b_id,score
0,client_1,5,5,2,520
1,client_1,10,10,2,110
2,client_1,10,10,9,230
3,client_4,10,10,2,110
4,client_4,10,10,9,230
13,client_3,4,4,3,340
14,client_3,1,1,2,300
15,client_3,1,1,4,150
16,client_3,1,1,7,220


In [77]:
basket_with_scores['row_number'] = basket_with_scores.groupby('client_id')["score"].rank(method="first", ascending=False)
basket_with_scores = basket_with_scores.sort_values(['client_id', 'row_number']).drop_duplicates(['client_id', 'b_id'])

C:\Users\Victor\AppData\Local\Temp\ipykernel_8808\3305768366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  basket_with_scores['row_number'] = basket_with_scores.groupby('client_id')["score"].rank(method="first", ascending=False)


In [78]:
basket_with_scores

,client_id,item_id,a_id,b_id,score,row_number
0,client_1,5,5,2,520,1.0
2,client_1,10,10,9,230,2.0
13,client_3,4,4,3,340,1.0
14,client_3,1,1,2,300,2.0
16,client_3,1,1,7,220,3.0
15,client_3,1,1,4,150,4.0
4,client_4,10,10,9,230,1.0
3,client_4,10,10,2,110,2.0


In [79]:
# ограничиваем число рекомендуемых товаров 4 штуками(по условию)
basket_with_scores = basket_with_scores.query('row_number <= 4')

In [80]:
recs_results = basket_with_scores.groupby('client_id')['b_id'].apply(list).reset_index()
recs_results

,client_id,b_id
0,client_1,"[2, 9]"
1,client_3,"[3, 2, 7, 4]"
2,client_4,"[9, 2]"


In [81]:
# фильтрация для бестселлеров - удаляем товары, которых нет в наличии
bestsellers = [x for x in bestsellers if x not in absent]
bestsellers

[7, 6, 4, 2, 8, 3, 9]

In [82]:
#Добавим бестселлеры
def add_bestsellers(rec_list, bestsellers, top_k):
    bestsellers_filtered = [i for i in bestsellers if i not in rec_list]
    return (rec_list + bestsellers_filtered)[:top_k]

recs_results['recs'] = recs_results['b_id'].apply(add_bestsellers, bestsellers=bestsellers, top_k=4)
recs_results = recs_results[['client_id', 'recs']] 
recs_results

,client_id,recs
0,client_1,"[2, 9, 7, 6]"
1,client_3,"[3, 2, 7, 4]"
2,client_4,"[9, 2, 7, 6]"


In [83]:
bestsellers_recs = (
    basket[['client_id']]
        .drop_duplicates()
)

bestsellers_recs = bestsellers_recs.where(~bestsellers_recs['client_id'].isin(recs_results['client_id'])).dropna()
bestsellers_recs['recs'] = bestsellers_recs.apply(lambda _: bestsellers[:4], axis=1)
bestsellers_recs

,client_id,recs
1,client_2,"[7, 6, 4, 2]"
4,client_5,"[7, 6, 4, 2]"


In [84]:
recs = pd.concat([recs_results, bestsellers_recs])
recs.sort_values('client_id')

,client_id,recs
0,client_1,"[2, 9, 7, 6]"
1,client_2,"[7, 6, 4, 2]"
1,client_3,"[3, 2, 7, 4]"
2,client_4,"[9, 2, 7, 6]"
4,client_5,"[7, 6, 4, 2]"
